In [2]:
def Make_tree2array(names_file):
    names = {}

    for line in names_file:
        if line == "\n":
            #print("empty row")
            continue

        data = [attr.strip() for attr in line.split("|")[0:2]]
        names[data[1]] = int(data[0])

    return names


def Nodes2Array(nodes_file):
    nodes = {}

    for line in nodes_file:
        if line == "\n":
            #print("empty row")
            continue

        data = [attr.strip() for attr in line.split("|")[0:3]]

        nodes[int(data[0])] = [int(data[1]), data[2]]
    return nodes


def Find_Value2Key_in_Names(Key, nodes, names, Taxon):
    if Key in sorted(names):
        Key_in_nodes = names[Key]
        if Key_in_nodes in sorted(nodes):
            if nodes[Key_in_nodes][1] == Taxon:
                #print("FOUND THE KEY")
                return Key
            # ----------------------------------------------------------------------------
            else:
                Find_Value2Key_in_Names(nodes[Key_in_nodes][0], nodes, names, Taxon)
        else:
            print("Key_in_nodes cannot be found.")
    else:
        #print("Key cannot be found.LABEL")
        return "UNKNOWN"
    #print("")



names_file = open("csado7/SILVA132/taxonomy/names.dmp", "r")
nodes_file = open("csado7/SILVA132/taxonomy/nodes.dmp", "r")

names = Make_tree2array(names_file)
nodes = Nodes2Array(nodes_file)


In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
import operator
import math

import Bio

import matplotlib.pyplot as plt

seqs = []
groups = []

transformed_group = []

class GroupedByGenus_reads():
    
    def __init__(self,Genus,ListOfGenus):
        self.Genus = Genus
        self.ListOfGenus = ListOfGenus
        self.reads = []
     
    def _add(self,element):
        self.ListOfGenus.append(element)
    
    def _read_list_extend(self, element):
        self.reads+=element
    
            
    def _add(self,element):
        self.ListOfGenus.append(element)

    def fill_reads(self,isFasta=True):
        asd=0
        for temp in self.ListOfGenus:
            #self.reads +=self.create_reads(temp.seq,temp.name)
            asd+=1
            if isFasta:
                self.reads.extend(create_reads_fasta(temp.seq,temp.name,self.Genus))
            else:
                self.reads.extend(create_reads(temp.seq,temp.name,self.Genus))
            
            
            
            if asd%5000==0:
                print(str(asd))
                 

class GroupedByGenus():
    
    def __init__(self,Genus,ListOfGenus):
        self.Genus = Genus
        self.ListOfGenus = ListOfGenus
        
    def _add(self,element):
        self.ListOfGenus.append(element)

count=int(0)
with open("csado7/SILVA132/SILVA/library/silva.fna") as input_handle:
    for record in SeqIO.parse(input_handle, "fasta"):
        
        
        
        #------------------------csak: Bacteria és 1300<= x hossz <= 1650----------------------------------
        
        if record.description.split(' ',1)[1].split(';')[0] != "Bacteria":
            continue
        
        if len(record.seq)<1300 and len(record.seq)>1650:
            continue
        
        #-------------------------------------------------------------------------
        count+=1
        
        #curr_genus = record.description.split(' ',1)[1].split(';')[-2]
        #curr_genus = record.description.split(' ',1)[1].split(';')[6]
        curr_genus = str(Find_Value2Key_in_Names(record.description.split(' ',1)[1].split(';')[-2], nodes, names, "genus"))
        #print(record.description.split(' ',1)[1].split(';')[-1])
        
        if type(curr_genus) != str:
            print(count)
            print("Ennyi")
            print(record.description)
            print(curr_genus)
            break
   
        change =0
            
        if count %15000==0:
            print(int(count)/510508*100)

        for i in range(0,len(transformed_group)):
            if transformed_group[i].Genus == curr_genus:
                #transformed_group[i].ListOfGenus.append(record)
                transformed_group[i]._add(record)
                change+=1
                
        if change ==0:
            gp = GroupedByGenus_reads(curr_genus,[])
            gp._add(record)
            #gp.Genus = curr_genus
            #gp.ListOfGenus.append(record)
            transformed_group.append(gp)
        
    
        #if not curr_genus in groups:
        #   groups.append(curr_genus) 
        #seqs.append(record)
        
print(count)
print(str(len(transformed_group)))

2.9382497433928556
5.876499486785711
8.814749230178567
11.752998973571422
14.691248716964278
17.629498460357134
20.567748203749993
23.505997947142845
26.4442476905357
29.382497433928556
32.320747177321415
35.25899692071427
38.197246664107126
41.135496407499986
44.07374615089283
47.01199589428569
49.95024563767855
52.8884953810714
55.82674512446426
58.76499486785711
61.70324461124997
64.64149435464283
67.57974409803568
70.51799384142853
73.4562435848214
76.39449332821425
79.3327430716071
82.27099281499997
431575
3269


In [7]:
import Bio

_asd=0
for s in transformed_group:
#for s in range(0,1):
    s_name=s.Genus
    
    if " " in s_name:
        s_name= s_name.replace(' ','_')

    if "-" in s_name:    
        s_name= s_name.replace('-','_')
    
    path= "MSA_score/Taxonomy_Genus_Just_Bacteria/"+s_name+".fasta"
    File_Genus = open(path, 'w')
    
    for r in s.ListOfGenus: 
        File_Genus.write(">"+r.description+"\n")
        File_Genus.write(str(r.seq)+"\n")
        _asd+=1
        
    File_Genus.close()
    
    
    
print(str(_asd))




431575


In [5]:

path= "Taxonomy_Genus_Just_Bacteria/Genus_group_name_WORKING.txt"
File_Genus = open(path, 'w')

k=0
for s in transformed_group:
    k+=1
    s_name=s.Genus
    if " " in s_name:
        s_name= s_name.replace(' ','_')

    if "-" in s_name:    
        s_name= s_name.replace('-','_')
    

    File_Genus.write(s_name+"\n")

        
File_Genus.close()
    
    
    
print(str(k))



3269


In [4]:

def create_reads(curr_seq,name,genus,a=30):
        return_string= []
        coverage = [0] * len(curr_seq)
        #--------------------

        c = int((len(curr_seq)+50)/(((len(curr_seq)-200)*a)/150))
        

        #--------------------
        x = len(curr_seq)

        short_reads = 0

        it_end =0

        Length_diff =[]

        while it_end < len(curr_seq):

            if it_end+150 > len(curr_seq):
                if it_end + 50 > len(curr_seq):
                    it_end = len(curr_seq)+1
                    break
                else:
                    read_end =len(curr_seq)
            else:
                read_end = it_end+150


            for delete in range(it_end,read_end):
                coverage[delete] += 1
                
            
            string_out = "@"+name + "|" +genus+"|"+ "S"+str(it_end)  + "E" + str(read_end) + "\n" \
                         + str(curr_seq[it_end:read_end]) + "\n"+ "+" \
            +"\n" +"H" * (read_end-it_end)+"\n"
            
            
            return_string.append(string_out)

            Length_diff.append(read_end-it_end)


            it_end += c
        return return_string
    
def create_reads_fasta(curr_seq,name,genus,a=30):
        return_string= []
        coverage = [0] * len(curr_seq)
        #--------------------

        c = int((len(curr_seq)+50)/(((len(curr_seq)-200)*a)/150))
        

        #--------------------
        x = len(curr_seq)

        short_reads = 0

        it_end =0

        Length_diff =[]

        while it_end < len(curr_seq):

            if it_end+150 > len(curr_seq):
                if it_end + 50 > len(curr_seq):
                    it_end = len(curr_seq)+1
                    break
                else:
                    read_end =len(curr_seq)
            else:
                read_end = it_end+150


            for delete in range(it_end,read_end):
                coverage[delete] += 1
                
            
            string_out = ">"+name + "|" +genus+"|"+ "S"+str(it_end)  + "E" + str(read_end) + "\n" \
                         + str(curr_seq[it_end:read_end]) + "\n"
            
            
            return_string.append(string_out)

            Length_diff.append(read_end-it_end)


            it_end += c
        return return_string

    
def print_(seq):
    print(seq)

print(len(transformed_group))

3269


In [5]:
import Bio


"""
for temp in groups:
    new_type = GroupedByGenus_reads(temp)
    transformed_group.append(new_type)
"""
#-----------------

#-----------------
t=transformed_group[0].ListOfGenus[0].seq

x_ =0

for el in transformed_group:
    x_+=1
    if el.Genus == "UNKNOWN":
        print("c:")
        continue
    
    #ez csak visszajelzés h hol jár
    if x_%50==0:
        print(str(len(el.ListOfGenus))+" , "+el.Genus+":  " + str(x_/len(transformed_group)*100)+"%-nál jár a beolvasás.")
    
    el.fill_reads()

    

_asd=0
for s in transformed_group:
#for s in range(0,1):

    s_name=s.Genus
    if " " in s_name:
        s_name= s_name.replace(' ','_')

    if "-" in s_name:    
        s_name= s_name.replace('-','_')
    
    
    path= "Genus_4_Bacteria_20221117/"+s_name+"_reads.fasta"
    File_Genus = open(path, 'w')
    
    for r in s.reads:    
        File_Genus.write(r)
        _asd+=1
    File_Genus.close()
    del s
print(str(_asd))



5000
5000
5000
5000
10000
15000
5000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
5000
245 , Luteimonas:  1.5295197308045274%-nál jár a beolvasás.
c:
1498 , Treponema:  3.059039461609055%-nál jár a beolvasás.
158 , Erysipelatoclostridium:  4.588559192413582%-nál jár a beolvasás.
484 , Ralstonia:  6.11807892321811%-nál jár a beolvasás.
229 , Marinifilum:  7.647598654022637%-nál jár a beolvasás.
213 , Kosakonia:  9.177118384827164%-nál jár a beolvasás.
5000
175 , Microvirga:  10.706638115631693%-nál jár a beolvasás.
7 , Allofrancisella:  12.23615784643622%-nál jár a beolvasás.
10 , Cyanothece PCC-7424:  13.765677577240746%-nál jár a beolvasás.
20 , Archangium:  15.295197308045275%-nál jár a beolvasás.
12 , Gimesia:  16.8247170388498%-nál jár a beolvasás.
128 , Planomicrobium:  18.354236769654328%-nál jár a beolvasás.
42 , Actinoallomurus:  19.883756500458855%-nál jár a beolvasás.
36 , Oleiphilus:  21.413276231263385%-nál jár a